# Circle Detection - Test
Given the picture of a noisy circle, determine the of the center **(x, y)** and the radius **r** of the circle.

Input:
- ResNet18  
  3-dimmension grayscale image
- Self-defined CNN  
  1-dimmension grayscale image

Output:  
- x: row of the circle center  
- y: column of the circle center  
- r: radius of the circle

**Please specify the image and model variables in the Part 0. Finally, run the entire python code.**

If model parameter file download is failed, please download the model parameter file (ex. model_resnet18.pt) from the [github](https://github.com/Dawson-ma/Circle-Detection) and upload to the colab.
(Colab left bar>files>upload to session storage)

## Part 0: Variables

Sample and model variables

In [17]:
num_samples = 2000
exist_dataset_path = None # If already have a dataset, please specify the path
model_arch = 'Resnet18' # Resnet18 or CNN
model_path = 'model_resnet18.pt' # 'model_resnet18.pt' or 'model_CNN.pt'
iou_thres = 0.95

In [18]:
# Fixed variables, please make sure to be same as training if modification is necessary
noise_level = 0.5
img_size = 100
min_radius = img_size // 10
max_radius = img_size // 2

## Part I: Packages Preparation and Helper Function Construction

### Import packages

In [19]:
# Package for mounting google drive
from google.colab import drive

# Package for loading data
import json

# Packages for model training
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torchvision.models import resnet18

# Packages for helper function
from typing import NamedTuple, Optional, Tuple, Generator
import numpy as np
from matplotlib import pyplot as plt
from skimage.draw import circle_perimeter_aa

In [32]:
!gdown --id 10bdgE1cyzuE737n2vCWcdobHw5tFSd2K --output model_resnet18.pt
!gdown --id 1bTGwja1qBQQq7ZKvfpskVO9saaVKkj0x --output model_CNN.pt

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=10bdgE1cyzuE737n2vCWcdobHw5tFSd2K
To: /content/model_resnet18.pt
100% 45.4M/45.4M [00:00<00:00, 65.0MB/s]


### Helper functions
Functions for generating noisy circle examples
- CircleParams: Circle parameters (row, col, radius)
- draw_circle: Draw a circle in a numpy array according to given row, col, and radius
- noisy_circle: Draw a circle in a numpy array, with normal noise.
- show_circle: Plot the given image
- generate_examples: Generator for noisy circle examples
- iou: Calculate the intersection over union of two circles

In [21]:
class CircleParams(NamedTuple):
    row: int
    col: int
    radius: int


def draw_circle(img: np.ndarray, row: int, col: int, radius: int) -> np.ndarray:
    """
    Draw a circle in a numpy array, inplace.
    The center of the circle is at (row, col) and the radius is given by radius.
    The array is assumed to be square.
    Any pixels outside the array are ignored.
    Circle is white (1) on black (0) background, and is anti-aliased.
    """
    rr, cc, val = circle_perimeter_aa(row, col, radius)
    valid = (rr >= 0) & (rr < img.shape[0]) & (cc >= 0) & (cc < img.shape[1])
    img[rr[valid], cc[valid]] = val[valid]
    return img


def noisy_circle(
    img_size: int, min_radius: float, max_radius: float, noise_level: float
) -> Tuple[np.ndarray, CircleParams]:
    """
    Draw a circle in a numpy array, with normal noise.
    """

    # Create an empty image
    img = np.zeros((img_size, img_size))

    radius = np.random.randint(min_radius, max_radius)

    # x,y coordinates of the center of the circle
    row, col = np.random.randint(img_size, size=2)

    # Draw the circle inplace
    draw_circle(img, row, col, radius)

    added_noise = np.random.normal(0.5, noise_level, img.shape)
    img += added_noise

    return img, CircleParams(row, col, radius)


def show_circle(img: np.ndarray):
    """Plot the given image"""
    fig, ax = plt.subplots()
    ax.imshow(img, cmap="gray")
    ax.set_title("Circle")
    plt.show()


def generate_examples(
    noise_level: float = 0.5,
    img_size: int = 100,
    min_radius: Optional[int] = None,
    max_radius: Optional[int] = None,
    dataset_path: str = "ds",
) -> Generator[Tuple[np.ndarray, CircleParams], None, None]:
    """
    Generate noise circle image examples
    """
    if not min_radius:
        min_radius = img_size // 10
    if not max_radius:
        max_radius = img_size // 2
    assert max_radius > min_radius, "max_radius must be greater than min_radius"
    assert img_size > max_radius, "size should be greater than max_radius"
    assert noise_level >= 0, "noise should be non-negative"

    params = (
        f"{noise_level=}, {img_size=}, {min_radius=}, {max_radius=}, {dataset_path=}"
    )
    print(f"Using parameters: {params}")
    while True:
        img, params = noisy_circle(
            img_size=img_size,
            min_radius=min_radius,
            max_radius=max_radius,
            noise_level=noise_level,
        )
        yield img, params


def iou(a: CircleParams, b: CircleParams) -> float:
    """Calculate the intersection over union of two circles"""
    r1, r2 = a.radius, b.radius
    d = np.linalg.norm(np.array([a.row, a.col]) - np.array([b.row, b.col]))
    if d > r1 + r2:
        return 0
    if d <= abs(r1 - r2):
        return 1
    r1_sq, r2_sq = r1**2, r2**2
    d1 = (r1_sq - r2_sq + d**2) / (2 * d)
    d2 = d - d1
    h1 = r1_sq * np.arccos(d1 / r1)
    h2 = d1 * np.sqrt(r1_sq - d1**2)
    h3 = r2_sq * np.arccos(d2 / r2)
    h4 = d2 * np.sqrt(r2_sq - d2**2)
    intersection = h1 + h2 + h3 + h4
    union = np.pi * (r1_sq + r2_sq) - intersection
    return intersection / union

## Part II: Data Processing
- Generate dataset
- Define custom dataset for noisy circle images

#### Generate dataset
Generate data for testing.

In [22]:
if not exist_dataset_path:
    # Build the data generator
    exGenerator = generate_examples(
        noise_level, img_size, min_radius, max_radius
    )

    # Generate data
    data = {}
    for i in range(num_samples):
        img, params = next(exGenerator)
        data[f"circle_{i:05d}"] = {
            "img": img.tolist(),
            "row": int(params.row),
            "col": int(params.col),
            "radius": int(params.radius),
        }
else:
    # Load dataset if existed
    with open(exist_dataset_path) as f:
        data = json.load(f)

Using parameters: noise_level=0.5, img_size=100, min_radius=10, max_radius=50, dataset_path='ds'


#### Custom Dataset
Define a dataset for circle images and *labels*

In [23]:
class CircleDataset(Dataset):
    def __init__(self, data: list, transform: transforms, RGB: bool):
        # Dataset data
        self.data = data
        self.transform = transform
        self.x, self.y = [], []

        # Iterate over the data
        for d in data:
            img = torch.FloatTensor(d["img"])

            # Transform 1 channel to 3 channels if required
            if RGB:
                img = img.expand(3, len(d["img"]), len(d["img"]))

            self.x.append(img)
            self.y.append(torch.FloatTensor([d["row"], d["col"], d["radius"]]))

    def __getitem__(self, idx):
        return self.transform(self.x[idx]), self.y[idx]

    def __len__(self):
        return len(self.data)

## Part III: Model Evaluation

### Model Architecture

In [24]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 64, 5)
        self.batchnorm1 = nn.BatchNorm2d(64)

        self.conv2 = nn.Conv2d(64, 256, 3)
        self.batchnorm2 = nn.BatchNorm2d(256)

        self.conv3 = nn.Conv2d(256, 256, 3)
        self.batchnorm3 = nn.BatchNorm2d(256)

        self.conv4 = nn.Conv2d(256, 32, 1)
        self.batchnorm4 = nn.BatchNorm2d(32)

        self.conv5 = nn.Conv2d(32, 4, 1)
        self.batchnorm5 = nn.BatchNorm2d(4)

        self.fc = nn.Linear(4*10*10, 3)
        self.act = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

    def forward(self, x):
        # 1 * 100 * 100
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = self.act(x)
        x = self.maxpool(x)

        # 64 * 48 * 48
        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = self.act(x)
        x = self.maxpool(x)

        # 256 * 23 * 23
        x = self.conv3(x)
        x = self.batchnorm3(x)
        x = self.act(x)
        x = self.maxpool(x)

        # 256 * 10 * 10
        x = self.conv4(x)
        x = self.batchnorm4(x)
        x = self.act(x)

        # 32 * 10 * 10
        x = self.conv5(x)
        x = self.batchnorm5(x)
        x = self.act(x)

        # 4 * 10 * 10
        B, C, H, W = x.shape
        x = x.view(-1, C * H * W)
        x = self.fc(x)

        return x

### Evaluation function

In [25]:
def evaluate(model, dataset, iou_thres, device):
    # Specify the device used to train the model
    model.to(device)

    # Loss criterion
    criterion = nn.MSELoss()

    # Dataloader
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=128, shuffle=False, drop_last=False
    )

    # Evaluate
    model.eval()

    # Define variables
    test_loss, test_iou, test_acc = 0.0, 0.0, 0.0

    # Iterate over test data.
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = criterion(labels, outputs)

        # statistics
        test_loss += loss.item()
        # Calculate accuracy and mean iou
        labels = labels.cpu()
        outputs = outputs.cpu()
        for truth, pred in zip(labels, outputs):
            sample_iou = iou(
                CircleParams(truth[0], truth[1], truth[2]),
                CircleParams(pred[0], pred[1], pred[2]),
            )
            test_acc += 1 if sample_iou >= iou_thres else 0
            test_iou += sample_iou

    print(f"Test evaluation:")
    print(f"iou threshold: {iou_thres}")
    print(
        f"loss: {test_loss:.4f} acc: {test_acc/len(dataset):.4f} iou: {test_iou/len(dataset)}"
    )

### Evaluation Process

Define data transformation

In [26]:
if model_arch == 'Resnet18':
    data_transforms = transforms.Compose(
        [transforms.Normalize(mean=[0.456, 0.456, 0.456], std=[0.225, 0.225, 0.225])]
    )
else:
    data_transforms = transforms.Compose(
        [transforms.Normalize(mean=[0.456], std=[0.225])]
    )

Construct dataset

In [27]:
# Extract data val and labels
data = [val for _, val in data.items()]

# Construct test dataset
dataset = CircleDataset(data, data_transforms, RGB=True if model_arch=='Resnet18' else False)

Build model

In [28]:
model = resnet18() if model_arch == 'Resnet18' else None
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256),
    nn.ReLU(),
    nn.Linear(256, 64),
    nn.ReLU(),
    nn.Linear(64, 16),
    nn.ReLU(),
    nn.Linear(16, 3),
)

Load the trained model

In [29]:
# Recognize the device
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"

In [33]:
if device == "cpu":
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
else:
    model.load_state_dict(torch.load(model_path))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Evaluate

In [34]:
evaluate(model, dataset, iou_thres, device)

Test evaluation:
iou threshold: 0.95
loss: 19.4413 acc: 0.9535 iou: 0.9880501627922058
